In [1]:
import re
import glob
import pandas as pd
import numpy as np
from datetime import datetime 
from itertools import product
from SetSimilaritySearch import all_pairs
# for CV
from sklearn.model_selection import GroupKFold
# process mining
import pm4py
from pm4py.objects.conversion.log import converter as log_converter
# miners
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
# performance metrics
from pm4py.algo.evaluation.replay_fitness import algorithm as replay_fitness_evaluator
from pm4py.algo.evaluation.precision import algorithm as precision_evaluator
from pm4py.algo.evaluation.generalization import algorithm as generalization_evaluator
from pm4py.algo.evaluation.simplicity import algorithm as simplicity_evaluator

In [2]:
def similarity_check(d=None, key_indices=None, min_cosine_sim=0.5):
    event_log_df = d.iloc[:, key_indices]
    event_log_df.columns.values[0] = 'case:concept:name'
    event_log_df.columns.values[1] = 'time:timestamp'
    event_log_df.columns.values[2] = 'concept:name'
    # get the sequence of activities by case_ids
    sets = event_log_df.groupby('case:concept:name')['concept:name']\
        .apply(list).apply(set).tolist()
    if len(sets) > 0:
        pairs = list(all_pairs(sets, similarity_func_name="cosine", similarity_threshold=0.1))
        #  print('pairs:', pairs)
        if len(pairs) == 0:
            cosine_sim = 0
        else:
            cosine_sim = np.mean([list(pairs[i])[2] for i in range(len(pairs))])
    else:
        cosine_sim = 0

    # print('cosine similarity:', cosine_sim)
    if cosine_sim <= min_cosine_sim:
        return False
    else:
        # if passed all preliminary tests, then return True
        return True

In [3]:
def measure_score(d=None, key_indices=None, miner='inductive_miner', metric='Buijs2014', n_splits=3):
    def discover_model(event_log, miner):
        # default parameters are used
        if miner == 'inductive_miner':
            net, im, fm = inductive_miner.apply(event_log,
                {pm4py.algo.discovery.inductive.variants.im.algorithm.Parameters.NOISE_THRESHOLD: 0.2},
                pm4py.algo.discovery.inductive.algorithm.Variants.IM)
        elif miner == 'heuristics_miner':
            net, im, fm = heuristics_miner.apply(event_log, {
                heuristics_miner.Variants.CLASSIC.value.Parameters.DEPENDENCY_THRESH: 0.5,
                heuristics_miner.Variants.CLASSIC.value.Parameters.AND_MEASURE_THRESH: 0.65,
                heuristics_miner.Variants.CLASSIC.value.Parameters.MIN_ACT_COUNT: 1,
                heuristics_miner.Variants.CLASSIC.value.Parameters.MIN_DFG_OCCURRENCES: 1,
                heuristics_miner.Variants.CLASSIC.value.Parameters.DFG_PRE_CLEANING_NOISE_THRESH: 0.05,
                heuristics_miner.Variants.CLASSIC.value.Parameters.LOOP_LENGTH_TWO_THRESH: 2})
        return net, im, fm

    def evaluate_score(event_log, net, im, fm, metric):
        if metric == 'fitness':
            score = replay_fitness_evaluator.apply(event_log, net, im, fm, 
                    variant=replay_fitness_evaluator.Variants.TOKEN_BASED)['log_fitness']
        elif metric == 'precision':
            score = precision_evaluator.apply(event_log, net, im, fm, 
                    variant=precision_evaluator.Variants.ETCONFORMANCE_TOKEN)
        elif metric == 'generalization':
            score = generalization_evaluator.apply(event_log, net, im, fm)
        elif metric == 'simplicity':
            score = simplicity_evaluator.apply(net)
        else:
            if metric == 'Buijs2014':
            # A paper "Quality dimensions in process discovery: The importance of fitness, 
            # precision, generalization and simplicity" proposed to calculate the following 
            # four metrics with giving 10 times more weight to replay fitness than 
            # the other three.
            # 10 x + 3x = 1 => x = 1 / 13
                weights = [10/13, 1/13, 1/13, 1/13]
            elif metric == 'average':
                weights = [0.25, 0.25, 0.25, 0.25]
            fitness = replay_fitness_evaluator.apply(event_log, net, im, fm, 
                    variant=replay_fitness_evaluator.Variants.TOKEN_BASED)['log_fitness']
            precision = precision_evaluator.apply(event_log, net, im, fm, 
                    variant=precision_evaluator.Variants.ETCONFORMANCE_TOKEN)
            #  fitness = replay_fitness_evaluator.apply(event_log, net, im, fm, 
                    #  variant=replay_fitness_evaluator.Variants.ALIGNMENT_BASED)['log_fitness']
            #  precision = precision_evaluator.apply(event_log, net, im, fm, 
                    #  variant=precision_evaluator.Variants.ALIGN_ETCONFORMANCE)
            generalization = generalization_evaluator.apply(event_log, net, im, fm)
            simplicity = simplicity_evaluator.apply(net)
            score = np.dot(weights, [fitness, precision, generalization, simplicity])
        return score
    
    def cross_validation(event_log_df, train, test):
        # dataset need be sorted by timestamp (https://pm4py.fit.fraunhofer.de/documentation#item-import-csv)
        #  print("%s %s" % (train, test))
        train_log_df = event_log_df.iloc[train]
        train_log_df = train_log_df.sort_values('time:timestamp')
        #  print('train_log_df')
        #  print(set(train_log_df['case:concept:name']))
        test_log_df = event_log_df.iloc[test]
        test_log_df = test_log_df.sort_values('time:timestamp')
        #  print('test_log_df')
        #  print(set(test_log_df['case:concept:name']))
        train_log = log_converter.apply(train_log_df, \
                                        variant=log_converter.Variants.TO_EVENT_LOG)
        test_log = log_converter.apply(test_log_df, \
                                       variant=log_converter.Variants.TO_EVENT_LOG)
    
        print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), \
              'discovering a process model...')
        #  print(train_log_df[['case:concept:name', 'time:timestamp', 'concept:name']])
        #  print(train_log_df.groupby('case:concept:name').size())
        #  print(set(train_log_df['case:concept:name']))
        #  print(set(train_log_df['time:timestamp']))
        #  print(set(train_log_df['concept:name']))
        net, im, fm = discover_model(train_log, miner)
        print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'process model discovered')
        score = evaluate_score(test_log, net, im, fm, metric)
        print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'score evaluated:', score)
        return score
    
    # assume the columns as case_id, timestamp, and activity from left
    event_log_df = d.iloc[:, key_indices]
    event_log_df.columns.values[0] = 'case:concept:name'
    event_log_df.columns.values[1] = 'time:timestamp'
    event_log_df.columns.values[2] = 'concept:name'
    # evaluate the goodness of such an assumption with CV
    gkf = GroupKFold(n_splits=n_splits)
    X = list(event_log_df.index)
    groups = list(event_log_df['case:concept:name'])
    try:
        scores = [cross_validation(event_log_df, train, test) \
              for train, test in gkf.split(X, groups=groups)]
        score = round(np.mean(scores), ndigits=3)
    except:
        score = 0
    print('final score:', score)
    return score

In [4]:
dataset_names = glob.glob('../datasets/*.csv', recursive=False)
n_rows = 1000
datasets = [pd.read_csv(d, nrows=n_rows) for d in dataset_names]

In [5]:
# try a dataset
dataset = datasets[5]

dataset

,question,dateFinished,dueDate,action_code,activityNameEN,planned,time:timestamp,monitoringResource,org:resource,activityNameNL,...,case:landRegisterID,case:parts,case:termName,case:startDate,case:requestComplete,case:IDofConceptCase,case:caseProcedure,case:Includes_subCases,case:endDatePlanned,dateStop
0,EMPTY,2014-01-10 00:00:00,2014-01-15 08:45:53+01:00,01_HOOFD_010,register submission date request,2014-01-13 08:45:53+01:00,2014-01-06 00:00:00+01:00,8492512,8492512,registratie datum binnenkomst aanvraag,...,6966675.0,Bouw,Termijn bezwaar en beroep 1,2014-01-06 00:00:00+01:00,True,10061287.0,NaN,NaN,NaN,NaN
1,False,2014-01-10 00:00:00,NaN,01_HOOFD_011,OLO messaging active,2014-01-13 08:55:48+01:00,2014-01-10 00:00:00+01:00,560604,8492512,OLO berichtenverkeer actief,...,6966675.0,Bouw,Termijn bezwaar en beroep 1,2014-01-06 00:00:00+01:00,True,10061287.0,NaN,NaN,NaN,NaN
2,True,2014-01-10 00:00:00,NaN,01_HOOFD_020,send confirmation receipt,2014-01-13 08:55:49+01:00,2014-01-10 00:00:00+01:00,560604,8492512,versturen ontvangstbevestiging,...,6966675.0,Bouw,Termijn bezwaar en beroep 1,2014-01-06 00:00:00+01:00,True,10061287.0,NaN,NaN,NaN,NaN
3,True,2014-01-10 00:00:00,NaN,03_GBH_005,applicant is stakeholder,2014-01-13 08:55:49+01:00,2014-01-10 00:00:00+01:00,560604,8492512,aanvrager is belanghebbende,...,6966675.0,Bouw,Termijn bezwaar en beroep 1,2014-01-06 00:00:00+01:00,True,10061287.0,NaN,NaN,NaN,NaN
4,False,2014-01-10 00:00:00,NaN,05_EIND_010,terminate on request,2014-01-13 08:55:49+01:00,2014-01-10 00:00:00+01:00,560604,8492512,beeindigen op verzoek,...,6966675.0,Bouw,Termijn bezwaar en beroep 1,2014-01-06 00:00:00+01:00,True,10061287.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,False,2014-02-20 00:00:00,NaN,01_HOOFD_490_2,decision date prior to decision,2014-02-21 11:48:12+01:00,2014-02-20 00:00:00+01:00,560604,560604,besluitdatum voorafgaand aan beschikking,...,10333172.0,Bouw,Termijn bezwaar en beroep 1,2014-02-11 00:00:00+01:00,True,NaN,NaN,NaN,NaN,NaN
996,False,2014-02-20 00:00:00,NaN,14_VRIJ_010,no permit needed or only notification needed,2014-02-17 10:03:50+01:00,2014-02-20 00:00:00+01:00,560604,560604,vergunningvrij of meldingplichtig,...,10333172.0,Bouw,Termijn bezwaar en beroep 1,2014-02-11 00:00:00+01:00,True,NaN,NaN,NaN,NaN,NaN
997,True,2014-02-20 00:00:00,NaN,01_HOOFD_110_0,create subcases completeness,2014-02-17 10:05:20+01:00,2014-02-20 00:00:00+01:00,560604,560604,deelzaken volledigheid aanmaken,...,10333172.0,Bouw,Termijn bezwaar en beroep 1,2014-02-11 00:00:00+01:00,True,NaN,NaN,NaN,NaN,NaN
998,EMPTY,2014-02-20 00:00:00,NaN,01_HOOFD_250_2,completed subcases content,2014-02-21 11:47:56+01:00,2014-02-20 00:00:00+01:00,560604,560604,deelzaken inhoudelijkheid afgerond,...,10333172.0,Bouw,Termijn bezwaar en beroep 1,2014-02-11 00:00:00+01:00,True,NaN,NaN,NaN,NaN,NaN


In [6]:
# we want to narrow down the candidates for key columns as much as possible
# A timestamp column should contain a digit in its values
# Hence we identify the columns that involve a digit
tmp = dataset.apply(lambda x: all([bool(re.search(r'\d', str(value))) for value in set(x)]), \
                    axis=0).tolist()
# get column indices as timestamp candidates
timestamp_cand = list(np.where(tmp)[0])
timestamp_cand

[1, 6, 7, 8, 10, 16, 17, 18, 22]

In [7]:
# case_id also should contain a digit (we leverage the timestamp_cand)
# in addition, the same case_id must appear multiple times as there cannot be only one activty per case
tmp = dataset.iloc[:, timestamp_cand] \
    .apply(lambda x: round(np.mean(x.value_counts()), ndigits=0) > 1, axis=0)
# get the indices of them
case_id_cand = list(np.array(timestamp_cand)[np.where(tmp)[0]])
case_id_cand

[1, 6, 7, 8, 10, 16, 17, 18, 22]

In [8]:
# activity 
# Likewise, the same activity must appear multiple times \
tmp = dataset.apply(lambda x: round(np.mean(x.value_counts()), ndigits=0) > 1, axis=0)
activity_cand = list(np.where(tmp)[0])
activity_cand

[0,
 1,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25]

In [9]:
# get every possible key columns' indices
# exclude any duplicates as we cannot select the same column for key indices (e.g. case_id and activity, etc.)
key_index_cand = [list(t) for t in product(case_id_cand, timestamp_cand, activity_cand) if len(set(t)) >= 3]

In [10]:
# as similar activity sequences have to be observed by multiple cases (otherwise we cannot identify a process flow)
# hence we rule out the cases where each sequence of activities do not share a common pattern
# in this regrad, we use cosine similarity to measure the overlaps between activities
tmp = [similarity_check(dataset, key_indices, min_cosine_sim=0.5) \
    for key_indices in key_index_cand]
combinations_to_try = np.array(key_index_cand)[np.array(tmp)]
# check how many candidates we try
len(combinations_to_try)

1041

In [ ]:
# after pre-filtering the candidates, try every combination to process discovery
# the intuition behind it is that the correct key column combination would result in
# a good score (e.g. fitness, precision, generalizability, simplicity)
scores = [measure_score(dataset, key_indices) for key_indices in combinations_to_try]

2022-07-13 06:33:47 discovering a process model...
2022-07-13 06:33:47 process model discovered


replaying log with TBR, completed variants ::   0%|          | 0/14 [00:00<?, ?it/s]

replaying log with TBR, completed variants ::   0%|          | 0/274 [00:00<?, ?it/s]

replaying log with TBR, completed variants ::   0%|          | 0/14 [00:00<?, ?it/s]

2022-07-13 06:33:50 score evaluated: 0.8633735564942978
2022-07-13 06:33:50 discovering a process model...
2022-07-13 06:33:50 process model discovered


...

replaying log with TBR, completed variants ::   0%|          | 0/296 [00:00<?, ?it/s]

replaying log with TBR, completed variants ::   0%|          | 0/24 [00:00<?, ?it/s]

2022-07-13 08:25:24 score evaluated: 0.8464786173730156
2022-07-13 08:25:24 discovering a process model...
2022-07-13 08:33:05 process model discovered


replaying log with TBR, completed variants ::   0%|          | 0/28 [00:00<?, ?it/s]

replaying log with TBR, completed variants ::   0%|          | 0/295 [00:00<?, ?it/s]

In [ ]:
h = dataset.columns.values.tolist()
correct_labels = [
        h.index('case:concept:name'),
        h.index('time:timestamp'),
        h.index('concept:name')
        ]
correct_labels

In [ ]:
measure_score(dataset, correct_labels)

In [ ]:
sorted(scores)

In [ ]:
# check if the correct tuple of key indices is in top-5 scores
top_5_cand_indices = sorted(range(len(scores)), key=lambda i: scores[i])[-5:]
correct_labels in np.array(combinations_to_try)[np.array(top_5_cand_indices)]